In [2]:

import findspark

In [3]:
findspark.init()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession \
        .builder \
        .appName("Streaming Word Count") \
        .master("local[3]") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 3) \
        .getOrCreate()

In [6]:
spark

In [7]:
# in windows : ncat -nl 9999
# in ubuntu : nc -nl 9999
lines_df = spark.readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", "12345") \
        .load()

In [8]:
lines_df.printSchema()


root
 |-- value: string (nullable = true)



In [10]:
from pyspark.sql.functions import *

In [11]:
lines_df = lines_df.coalesce(1)
words_df = lines_df.select(expr("explode(split(value,' ')) as word"))
counts_df = words_df.groupBy("word").count()

# word
# -----
# car 
# car 
# car 

# river 
# river


In [12]:
 word_count_query = counts_df.writeStream \
        .format("console") \
        .outputMode("complete") \
        .option("checkpointLocation", "d:\\data\\chk-point-dir") \
        .start()

In [ ]:
# logger.info("Listening to localhost:9999")
word_count_query.awaitTermination()